In [6]:
# 의사록(MPB)

import requests, os
from bs4 import BeautifulSoup

# 각자 파일 경로
my_local_pdf_dir_path = 'secret_MPB_minutes'
# 디렉토리 생성
os.makedirs(f'./{my_local_pdf_dir_path}', exist_ok=True)

for i in range(1, 40):
    b_url = "https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761"
    params = {
        'pageIndex' : i
    }

    response = requests.get(b_url, params=params)
    soup = BeautifulSoup(response.content)

    li_list = []
    li = soup.find('div', class_='bdLine type2').find('ul').find('li')
    li_li = li.find_next_siblings('li', li)
    li_list.append(li)
    li_list.extend(li_li)

    print(i)

    for x in range(0, len(li_list)):
        try:
            link = li_list[x].find('div', class_='fileGoupBox').find('ul').find('li').find('a').attrs['href']
            name_ = li_list[x].find('div', class_='fileGoupBox').find('ul').find('li').find('a').getText().strip()
            title = li_list[x].find('div', class_='row').find('span').find('a').find('span').find('span').text
            url2 = 'http://www.bok.or.kr' + link
            file_res = requests.get(url2)
            if name_[-4:]=='.hwp':
                link = li_list[x].find('div', class_='fileGoupBox').find('ul').find_all('li')[1].find('a').attrs['href']
                url2 = 'http://www.bok.or.kr' + link
                file_res = requests.get(url2)
                with open('./{}/{}.pdf'.format(my_local_pdf_dir_path, title), 'wb') as f:
                    f.write(file_res.content)
            else:
                with open('./{}/{}.pdf'.format(my_local_pdf_dir_path, title), 'wb') as f:
                    f.write(file_res.content)

        except:
            try:
                link2 = li_list[x].find('div').find('div').find('a').attrs['href']
                name_ = li_list[x].find('div', class_='fileGoupBox').find('ul').find('li').find('a').getText().strip()
                title = li_list[x].find('div', class_='row').find('span').find('a').find('span').find('span').text
                url3 = 'http://www.bok.or.kr' + link2
                file_res2 = requests.get(url3)
                if name_[-4:]=='.hwp':
                    link2 = li_list[x].find('div').find('div').find_all('a')[1].attrs['href']
                    url3 = 'http://www.bok.or.kr' + link2
                    with open('./{}/{}.pdf'.format(my_local_pdf_dir_path, title), 'wb') as f:
                        f.write(file_res.content)
                else:
                    with open('./{}/{}.pdf'.format(my_local_pdf_dir_path, title), 'wb') as f:
                        f.write(file_res.content)
            except:
                break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [8]:
# MPB minutes pdf download code
import requests, urllib, os
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm


# 각자 파일 경로
my_local_pdf_dir_path = 'secret_MPB_minutes'
# 디렉토리 생성
os.makedirs(f'./{my_local_pdf_dir_path}', exist_ok=True)
# b_url: base url, {page}: page number
b_url = "http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={page}"

for i in tqdm(range(1, 40)):
    response = requests.get(b_url.format(i))
    soup = BeautifulSoup(response.content)

    # file_name_list: 고유한 파일명을 위한 파일명 리스트 추출
    file_name_list = soup.find_all('span', class_='col m10 s10 x9 ctBx')
    # tmp: 크롤링할 파일 url 목록 추출
    tmp = soup.find_all('div', class_='fileGoupBox')
    
    # 2개의 리스트를 이용하기에 range 사용
    for j in range(len(tmp)):
        # 현재 file의 파일명 할당
        file_name = file_name_list[j].find('span', class_='titlesub').getText().strip()
        # 현재 file 위해 사용할 태그
        for_download = tmp[j]
        # naem_: 파일 확장자 확인 위한 변수
        name_ = for_download.find('li').find('a').getText().strip()
        if name_[-4:]=='.hwp':
            url = ('http://www.bok.or.kr/'+for_download.find_all('li')[1].find('a')['href'])
            with urllib.request.urlopen(url) as web_file, open(f'./{my_local_pdf_dir_path}/{file_name}.pdf', 'wb') as local_file:
                local_file.write(web_file.read())
        else:
            url = ('http://www.bok.or.kr/'+for_download.find('li').find('a')['href'])
            with urllib.request.urlopen(url) as web_file, open(f'./{my_local_pdf_dir_path}/{file_name}.pdf', 'wb') as local_file:
                local_file.write(web_file.read())

  0%|          | 0/39 [00:00<?, ?it/s]

KeyError: 'page'

In [9]:
# 채권 분석 리포트 https://finance.naver.com/research/debenture_list.naver
import time
from datetime import datetime
import requests
from bs4 import BeautifulSoup

for i in range (1, 198):
    bond_url = "https://finance.naver.com/research/debenture_list.naver"
    params = {
        'page' : i
    }
    headers = {'authority' : 'finance.naver.com',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36' }
   

    response = requests.get(bond_url, params=params, headers=headers)
    soup = BeautifulSoup(response.content)
    bond = soup.select('#contentarea_left .file')
    bond_list = []
    for x in range(len(bond)):
        if bond[x].find_previous_sibling('td').text == '유안타증권':
            try:    
                bond_list.append([soup.select("#contentarea_left .file a[href$='.pdf']")[x].attrs['href'], bond[x].find_previous_sibling('td').find_previous_sibling('td').text])
            except:
                print("통과")

    for y in bond_list:
        file_res = requests.get(y[0])
        title = y[-1]
        try:
            with open('{}.pdf'.format(title), 'wb') as f:
                f.write(file_res.content)
        except:
            print('통과')


    
            

통과
통과


1. 월별 기사수 '금리'라는 단어 포함
2. 각 신문사별 기사수
3. 채권분석데이터수

In [6]:
# NNC(NaverNewsCrawling)
import time
import datetime
import requests, os
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

date_date = pd.date_range('2012-12-30', '2023-08-31', freq='W') #2013년부터 2023년까지 뉴스데이터 크롤링
date_role = [date_date[:130],date_date[130:260],date_date[260:390],date_date[390:520],date_date[520:]]

dir_path = './secret_news_crawling'
os.makedirs(dir_path, exist_ok=True)

for date in tqdm(date_role[0]):  #자신의 번호로 인덱싱
    df = pd.DataFrame(columns =['신문사', '제목', '내용', '날짜'])
    DateStart = date.strftime('%Y-%m-%d')
    DateEnd = (date+datetime.timedelta(6)).strftime('%Y-%m-%d')
    last_url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q=%B1%DD%B8%AE&x=20&y=14&sm=all.basic&pd=4&stDateStart={DateStart}&stDateEnd={DateEnd}&page=1"
    headers = {'authority' : 'finance.naver.com',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
    response = requests.get(last_url, headers=headers)
    soup = BeautifulSoup(response.content)
    last_page = soup.select('.pgRR a')[0]['href'].split('=')[-1] #마지막 페이지 가져오기
    print(last_page)
    for i in tqdm(range(1, int(last_page)+1)):   # 여기서 마지막 페이지까지 반복문
        news_url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q=%B1%DD%B8%AE&x=20&y=14&sm=all.basic&pd=4&stDateStart={DateStart}&stDateEnd={DateEnd}&page={i}"
        headers = {'authority' : 'finance.naver.com',
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
        response = requests.get(news_url, headers=headers)
        soup = BeautifulSoup(response.content)
        news = soup.select('.thumb a' )
        magazine = soup.select('.articleSummary .press') #신문사 데이터
        newsdate = soup.select('.articleSummary .wdate') # 날짜 데이터

        news_link = []
        magazine_name = []
        dates = []
        for i in range(len(news)):
            news_link.append('https://finance.naver.com' + news[i].attrs['href'].replace('amp;', ''))  #amp;제거해서 링크활성화
            magazine_name.append(magazine[i].text)
            dates.append(newsdate[i])

        total = []
        for index, j in enumerate(news_link):
            news_resp = requests.get(j, headers=headers)
            news_soup = BeautifulSoup(news_resp.content)
            try:
                title = news_soup.select('.article_info h3')[0].text.strip()
                content = news_soup.select('#content')[-1].text.strip()
            except:
                print(j)
                continue
            total.append([magazine_name[index],title, content, newsdate[index].text.lstrip()[:10]])
        
        df_temp = pd.DataFrame(total, columns = ['신문사', '제목', '내용', '날짜'])  #모든 리스트를 데이터프레임으로 전환
        df = pd.concat([df, df_temp], ignore_index = True)
    df.to_csv(f'{dir_path}/{DateStart}_{DateEnd}.csv', encoding='utf-8') # 1주일 간격으로 크롤링 데이터 저장    

  0%|          | 0/130 [00:00<?, ?it/s]

43


  1%|          | 1/130 [03:29<7:29:35, 209.11s/it]

79


  2%|▏         | 2/130 [08:39<9:33:20, 268.75s/it]

...

121


https://finance.naver.com/news/news_read.naver?article_id=0003512181&office_id=009&mode=search&query=금리&page=48


https://finance.naver.com/news/news_read.naver?article_id=0001486201&office_id=421&mode=search&query=금리&page=66


https://finance.naver.com/news/news_read.naver?article_id=0007674380&office_id=001&mode=search&query=금리&page=121


100%|██████████| 130/130 [13:17:11<00:00, 367.94s/it]


In [7]:
# BAC(Bond Analysis Crawling)
import time
import datetime
import requests, os
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
# '채권보고서' 라는 이름으로 검색을 해서 나온 기사 가져오기
date_date = pd.date_range('2012-12-30', '2023-08-31', freq='30D') #2013년부터 2023년까지 채권보고서 기사 크롤링(해당 기간동안 130개)
date_role = [date_date[:26],date_date[26:52],date_date[52:78],date_date[78:104],date_date[104:]]
dir_path = './secret_bond_crawling'
os.makedirs(dir_path, exist_ok=True) # mkdir과 달리 이미 해당이름의 폴더가 있어도 에러가 나지 않는다.
for date in tqdm(date_role[0]):  #자신의 번호로 인덱싱
    df = pd.DataFrame(columns =['신문사', '제목', '내용', '날짜'])
    DateStart = date.strftime('%Y-%m-%d')
    DateEnd = (date+datetime.timedelta(29)).strftime('%Y-%m-%d')
    bond_last_url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q=%C3%A4%B1%C7+%BA%B8%B0%ED%BC%AD&x=21&y=6&sm=all.basic&pd=4&stDateStart={DateStart}&stDateEnd={DateEnd}&page=1"
    headers = {'authority' : 'finance.naver.com',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
    response = requests.get(last_url, headers=headers)
    soup = BeautifulSoup(response.text)
    last_page = soup.select('.pgRR a')[0]['href'].split('=')[-1] #마지막 페이지 가져오기
    print(last_page)
    for i in tqdm(range(1, int(last_page)+1)):   # 여기서 마지막 페이지까지 반복문
        bond_url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q=%C3%A4%B1%C7+%BA%B8%B0%ED%BC%AD&x=21&y=6&sm=all.basic&pd=4&stDateStart={DateStart}&stDateEnd={DateEnd}&page={i}"
        headers = {'authority' : 'finance.naver.com',
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
        response = requests.get(news_url, headers=headers)
        soup = BeautifulSoup(response.text)
        news = soup.select('.thumb a' )
        magazine = soup.select('.articleSummary .press') #신문사 데이터
        newsdate = soup.select('.articleSummary .wdate') # 날짜 데이터
        news_link = []
        magazine_name = []
        dates = []
        for i in range(len(news)):
            news_link.append('https://finance.naver.com' + news[i].attrs['href'].replace('amp;', ''))  #amp;제거해서 링크활성화
            magazine_name.append(magazine[i].text)
            dates.append(newsdate[i])
        total = []
        for index, j in enumerate(news_link):
            news_resp = requests.get(j, headers=headers)
            news_soup = BeautifulSoup(news_resp.text)
            try:
                title = news_soup.select('.article_info h3')[0].text.strip()
                content = news_soup.select('#content')[-1].text.strip()
            except:
                print(j)
                continue
            total.append([magazine_name[index],title, content, newsdate[index].text.lstrip()[:10]])
        df_temp = pd.DataFrame(total, columns = ['신문사', '제목', '내용', '날짜'])  #모든 리스트를 데이터프레임으로 전환
        df = pd.concat([df, df_temp], ignore_index = True)
    df.to_csv(f'{dir_path}/{DateStart}_{DateEnd}.csv', encoding='utf-8') # 1주일 간격으로 크롤링 데이터 저장


  0%|          | 0/556 [00:00<?, ?it/s]

84


In [10]:
import pandas as pd
news = pd.read_csv('2023-08-20_2023-08-27.csv')
news

FileNotFoundError: [Errno 2] No such file or directory: '2023-08-20_2023-08-27.csv'

In [11]:
import time
import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

today = datetime.datetime.today()

DateStart = (today-datetime.timedelta(7)).strftime('%Y-%m-%d')
DateEnd = today.strftime('%Y-%m-%d')


last_url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q=%B1%DD%B8%AE&x=20&y=14&sm=all.basic&pd=4&stDateStart={DateStart}&stDateEnd={DateEnd}&page=1"
headers = {'authority' : 'finance.naver.com',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
response = requests.get(last_url, headers=headers)
soup = BeautifulSoup(response.content)
news = soup.select('.pgRR a')[0]['href'][-3:]
news

'134'

In [15]:
# 채권 기사
# BAC(Bond Analysis Crawling)
import time
import datetime
import requests, os
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
# '채권보고서' 라는 이름으로 검색을 해서 나온 기사 가져오기
date_date = pd.date_range('2012-12-30', '2023-08-31', freq='30D') #2013년부터 2023년까지 채권보고서 기사 크롤링(해당 기간동안 130개)
date_role = [date_date[:26],date_date[26:52],date_date[52:78],date_date[78:104],date_date[104:]]
dir_path = './secret_bond_crawling'
os.makedirs(dir_path, exist_ok=True) # mkdir과 달리 이미 해당이름의 폴더가 있어도 에러가 나지 않는다.
for date in tqdm(date_role[2]):  #자신의 번호로 인덱싱
    df = pd.DataFrame(columns =['신문사', '제목', '내용', '날짜'])
    DateStart = date.strftime('%Y-%m-%d')
    DateEnd = (date+datetime.timedelta(29)).strftime('%Y-%m-%d')
    bond_last_url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q=%C3%A4%B1%C7+%BA%B8%B0%ED%BC%AD&x=21&y=6&sm=all.basic&pd=4&stDateStart={DateStart}&stDateEnd={DateEnd}&page=1"
    headers = {'authority' : 'finance.naver.com',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
    response = requests.get(last_url, headers=headers)
    soup = BeautifulSoup(reponse.text)
    last_page = soup.select('.pgRR a')[0]['href'].split('=')[-1] #마지막 페이지 가져오기
    print(last_page)
    for i in tqdm(range(1, int(last_page)+1)):   # 여기서 마지막 페이지까지 반복문
        bond_url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q=%C3%A4%B1%C7+%BA%B8%B0%ED%BC%AD&x=21&y=6&sm=all.basic&pd=4&stDateStart={DateStart}&stDateEnd={DateEnd}&page={i}"
        headers = {'authority' : 'finance.naver.com',
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
        response = requests.get(news_url, headers=headers)
        soup = BeautifulSoup(reponse.text)
        news = soup.select('.thumb a' )
        magazine = soup.select('.articleSummary .press') #신문사 데이터
        newsdate = soup.select('.articleSummary .wdate') # 날짜 데이터
        news_link = []
        magazine_name = []
        dates = []
        for i in range(len(news)):
            news_link.append('https://finance.naver.com' + news[i].attrs['href'].replace('amp;', ''))  #amp;제거해서 링크활성화
            magazine_name.append(magazine[i].text)
            dates.append(newsdate[i])
        total = []
        for index, j in enumerate(news_link):
            news_resp = requests.get(j, headers=headers)
            news_soup = BeautifulSoup(news_resp.text)
            try:
                title = news_soup.select('.article_info h3')[0].text.strip()
                content = news_soup.select('#content')[-1].text.strip()
            except:
                print(j)
                continue
            total.append([magazine_name[index],title, content, newsdate[index].text.lstrip()[:10]])
        df_temp = pd.DataFrame(total, columns = ['신문사', '제목', '내용', '날짜'])  #모든 리스트를 데이터프레임으로 전환
        df = pd.concat([df, df_temp], ignore_index = True)
    df.to_csv(f'{dir_path}/{DateStart}_{DateEnd}.csv', encoding='utf-8') # 1주일 간격으로 크롤링 데이터 저장

  0%|          | 0/26 [00:00<?, ?it/s]

  0%|          | 0/26 [00:00<?, ?it/s]


NameError: name 'reponse' is not defined